## Imports

In [97]:
import os
import sys
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram, get_window

## Process

In [4]:
def stft (d,fs,winlen,nfft):
    noverlap=fs/100
    nperseg=winlen
    win = get_window('hamming',winlen)
    freqbin,tbin,spec=spectrogram(x=d,fs=fs,noverlap=noverlap,nperseg=nperseg,nfft=nfft,window=win)
    spec=(spec*1000)+1
    spec=np.real(np.log(spec))
    spec[spec<0]=0
    return spec, freqbin, tbin

In [5]:
wav_path = os.path.join(os.getcwd(), 'Data//Recordings//test//Sound', 'MAPS_ISOL_CH0.1_F_AkPnBcht.wav')
fs, data = wavfile.read(wav_path)

In [6]:
data = data[:,1]

In [7]:
# Define frequencies for notes up to 100 (first harmonic of 88th note)
Notes = np.zeros(100)
Notes[0] = 27.5
for i in range(99):
    Notes[i+1] = Notes[i]*(np.power(2,1/12))

In [8]:
winlen1=2048
nfft1=2048

winlen2=8192
nfft2=8192

In [9]:
spec1,freqbin1,tbin1 = stft(d=data,fs=fs,winlen=winlen1,nfft=nfft1)
spec2,freqbin2,tbin2 = stft(d=data,fs=fs,winlen=winlen2,nfft=nfft2)

In [10]:
tbin1.shape

(599,)

In [13]:
Q.shape

(599, 100)

In [91]:
Q=np.zeros([tbin1.shape[0],Notes.shape[0]])
for i in range(spec1.shape[1]):
    Z=spec1[:,i]
    for j in range(Notes.shape[0]-1):
        if j==0:
            j=1
        
        f1=Notes[j-1]
        f2=Notes[j+1]
        wb1 = np.where(f1>=freqbin1)[0]
        wb2 = np.where(f2<=freqbin1)[0]
        wb=np.intersect1d(wb1,wb2)
        bn=np.argmin(np.abs(freqbin1-Notes[j]))
        
        if ((wb.shape[0]==0) or wb.shape[0]==1):
            Q[i,j]=Z[j]
            
        else:
            x1 = wb[0:np.where(wb==bn)[0][0]+1]
            x2 = wb[np.where(wb==bn)[0][0]:]
            y1 = (x1-wb[0])/(x1.shape[0]-1)
            y2 = 1-(x2-bn)/(x2.shape[0]-1)
            y1[np.isnan(y1)]=1
            y2[np.isnan(y2)]=1
            Q[i,j]= (np.matmul(Z[x1],y1)+np.matmul(Z[x2],y2)-Z[bn])/(np.sum(y1)+np.sum(y2))